# Sales Funnel Analysis

This notebook analyzes the sales funnel to identify bottlenecks and conversion patterns.

## Objectives
1. Analyze stage-level drop-offs in the sales funnel
2. Compare conversion rates across industries and company sizes
3. Identify top-performing products, agents, and regions
4. Analyze sales cycle duration patterns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# Load the master dataset
df = pd.read_csv('../data/master_dataset.csv')
df['engage_date'] = pd.to_datetime(df['engage_date'])
df['close_date'] = pd.to_datetime(df['close_date'])

print(f"Dataset shape: {df.shape}")
df.head()

## 1. Funnel Overview & Stage Distribution

In [ ]:
# Stage distribution
stage_counts = df['deal_stage'].value_counts()
stage_pct = (stage_counts / len(df) * 100).round(1)

funnel_summary = pd.DataFrame({
    'Count': stage_counts,
    'Percentage': stage_pct
})

# Reorder stages logically
stage_order = ['Prospecting', 'Engaging', 'Won', 'Lost']
funnel_summary = funnel_summary.reindex(stage_order)

print("=== FUNNEL STAGE DISTRIBUTION ===")
print(funnel_summary)

In [ ]:
# Visualize funnel
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
colors = ['#3498db', '#f39c12', '#27ae60', '#e74c3c']
axes[0].bar(stage_order, funnel_summary['Count'], color=colors)
axes[0].set_title('Opportunities by Deal Stage', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Deal Stage')
axes[0].set_ylabel('Number of Opportunities')
for i, (stage, row) in enumerate(funnel_summary.iterrows()):
    axes[0].annotate(f"{row['Count']:,}\n({row['Percentage']}%)", 
                     xy=(i, row['Count']), ha='center', va='bottom', fontsize=10)

# Pie chart for closed deals only
closed_df = df[df['deal_stage'].isin(['Won', 'Lost'])]
closed_counts = closed_df['deal_stage'].value_counts()
axes[1].pie(closed_counts, labels=closed_counts.index, autopct='%1.1f%%', 
            colors=['#27ae60', '#e74c3c'], startangle=90)
axes[1].set_title('Win/Loss Distribution (Closed Deals)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../visuals/01_funnel_overview.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate overall win rate
closed_deals = df[df['deal_stage'].isin(['Won', 'Lost'])]
won_deals = df[df['deal_stage'] == 'Won']
lost_deals = df[df['deal_stage'] == 'Lost']

overall_win_rate = len(won_deals) / len(closed_deals) * 100

print(f"=== OVERALL CONVERSION METRICS ===")
print(f"Total Opportunities: {len(df):,}")
print(f"Closed Deals: {len(closed_deals):,}")
print(f"Won Deals: {len(won_deals):,}")
print(f"Lost Deals: {len(lost_deals):,}")
print(f"\nOverall Win Rate: {overall_win_rate:.1f}%")
print(f"Total Revenue: ${won_deals['close_value'].sum():,.0f}")
print(f"Average Deal Size: ${won_deals['close_value'].mean():,.0f}")

## 2. Conversion Rates by Industry (Sector)

In [ ]:
# Win rate by sector
sector_analysis = closed_deals.groupby('sector').agg(
    total_deals=('deal_stage', 'count'),
    won_deals=('is_won', 'sum'),
    total_revenue=('close_value', 'sum'),
    avg_deal_value=('close_value', lambda x: x[x > 0].mean())
).reset_index()

sector_analysis['win_rate'] = (sector_analysis['won_deals'] / sector_analysis['total_deals'] * 100).round(1)
sector_analysis = sector_analysis.sort_values('win_rate', ascending=False)

print("=== WIN RATE BY INDUSTRY SECTOR ===")
print(sector_analysis.to_string(index=False))

In [ ]:
# Visualize sector performance
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Win rate by sector
sector_sorted = sector_analysis.sort_values('win_rate', ascending=True)
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(sector_sorted)))
axes[0].barh(sector_sorted['sector'], sector_sorted['win_rate'], color=colors)
axes[0].set_xlabel('Win Rate (%)')
axes[0].set_title('Win Rate by Industry Sector', fontsize=14, fontweight='bold')
axes[0].axvline(x=overall_win_rate, color='red', linestyle='--', label=f'Overall: {overall_win_rate:.1f}%')
axes[0].legend()

# Revenue by sector
sector_by_rev = sector_analysis.sort_values('total_revenue', ascending=True)
axes[1].barh(sector_by_rev['sector'], sector_by_rev['total_revenue'] / 1000, color='#3498db')
axes[1].set_xlabel('Total Revenue ($K)')
axes[1].set_title('Revenue by Industry Sector', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../visuals/02_sector_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Conversion Rates by Company Size

In [ ]:
# Win rate by company size
size_analysis = closed_deals.groupby('company_size').agg(
    total_deals=('deal_stage', 'count'),
    won_deals=('is_won', 'sum'),
    total_revenue=('close_value', 'sum'),
    avg_deal_value=('close_value', lambda x: x[x > 0].mean())
).reset_index()

size_analysis['win_rate'] = (size_analysis['won_deals'] / size_analysis['total_deals'] * 100).round(1)

# Order by size
size_order = ['Small', 'Medium', 'Large', 'Enterprise', 'Corporate']
size_analysis['size_order'] = size_analysis['company_size'].map({s: i for i, s in enumerate(size_order)})
size_analysis = size_analysis.sort_values('size_order').drop('size_order', axis=1)

print("=== WIN RATE BY COMPANY SIZE ===")
print(size_analysis.to_string(index=False))

In [ ]:
# Visualize company size performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Win rate by size
x_pos = range(len(size_analysis))
axes[0].bar(x_pos, size_analysis['win_rate'], color='#2ecc71')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(size_analysis['company_size'], rotation=45, ha='right')
axes[0].set_ylabel('Win Rate (%)')
axes[0].set_title('Win Rate by Company Size', fontsize=14, fontweight='bold')
axes[0].axhline(y=overall_win_rate, color='red', linestyle='--', label=f'Overall: {overall_win_rate:.1f}%')
axes[0].legend()

# Avg deal value by size
axes[1].bar(x_pos, size_analysis['avg_deal_value'], color='#9b59b6')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(size_analysis['company_size'], rotation=45, ha='right')
axes[1].set_ylabel('Average Deal Value ($)')
axes[1].set_title('Average Deal Value by Company Size', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../visuals/03_company_size_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Product Performance Analysis

In [ ]:
# Win rate by product
product_analysis = closed_deals.groupby(['product', 'series']).agg(
    total_deals=('deal_stage', 'count'),
    won_deals=('is_won', 'sum'),
    total_revenue=('close_value', 'sum'),
    avg_deal_value=('close_value', lambda x: x[x > 0].mean())
).reset_index()

product_analysis['win_rate'] = (product_analysis['won_deals'] / product_analysis['total_deals'] * 100).round(1)
product_analysis = product_analysis.sort_values('total_revenue', ascending=False)

print("=== PRODUCT PERFORMANCE ===")
print(product_analysis.to_string(index=False))

In [ ]:
# Visualize product performance
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Revenue by product
prod_sorted = product_analysis.sort_values('total_revenue', ascending=True)
colors = ['#e74c3c' if s == 'MG' else '#3498db' if s == 'GTX' else '#f39c12' for s in prod_sorted['series']]
axes[0].barh(prod_sorted['product'], prod_sorted['total_revenue'] / 1000, color=colors)
axes[0].set_xlabel('Total Revenue ($K)')
axes[0].set_title('Revenue by Product', fontsize=14, fontweight='bold')

# Win rate by product
prod_by_wr = product_analysis.sort_values('win_rate', ascending=True)
colors_wr = ['#e74c3c' if s == 'MG' else '#3498db' if s == 'GTX' else '#f39c12' for s in prod_by_wr['series']]
axes[1].barh(prod_by_wr['product'], prod_by_wr['win_rate'], color=colors_wr)
axes[1].set_xlabel('Win Rate (%)')
axes[1].set_title('Win Rate by Product', fontsize=14, fontweight='bold')
axes[1].axvline(x=overall_win_rate, color='black', linestyle='--', label=f'Overall: {overall_win_rate:.1f}%')
axes[1].legend()

plt.tight_layout()
plt.savefig('../visuals/04_product_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Sales Team Performance

In [ ]:
# Regional performance
regional_analysis = closed_deals.groupby('regional_office').agg(
    total_deals=('deal_stage', 'count'),
    won_deals=('is_won', 'sum'),
    total_revenue=('close_value', 'sum'),
    agent_count=('sales_agent', 'nunique')
).reset_index()

regional_analysis['win_rate'] = (regional_analysis['won_deals'] / regional_analysis['total_deals'] * 100).round(1)
regional_analysis['revenue_per_agent'] = (regional_analysis['total_revenue'] / regional_analysis['agent_count']).round(0)

print("=== REGIONAL PERFORMANCE ===")
print(regional_analysis.to_string(index=False))

In [ ]:
# Top sales agents
agent_analysis = closed_deals.groupby(['sales_agent', 'manager', 'regional_office']).agg(
    total_deals=('deal_stage', 'count'),
    won_deals=('is_won', 'sum'),
    total_revenue=('close_value', 'sum')
).reset_index()

agent_analysis['win_rate'] = (agent_analysis['won_deals'] / agent_analysis['total_deals'] * 100).round(1)

# Filter to agents with at least 50 closed deals for statistical significance
top_agents = agent_analysis[agent_analysis['total_deals'] >= 50].sort_values('win_rate', ascending=False).head(10)

print("=== TOP 10 SALES AGENTS (by Win Rate, min 50 deals) ===")
print(top_agents.to_string(index=False))

In [ ]:
# Visualize regional performance
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Win rate by region
colors = ['#3498db', '#2ecc71', '#e74c3c']
axes[0].bar(regional_analysis['regional_office'], regional_analysis['win_rate'], color=colors)
axes[0].set_ylabel('Win Rate (%)')
axes[0].set_title('Win Rate by Region', fontsize=12, fontweight='bold')
axes[0].axhline(y=overall_win_rate, color='black', linestyle='--')

# Revenue by region
axes[1].bar(regional_analysis['regional_office'], regional_analysis['total_revenue'] / 1000, color=colors)
axes[1].set_ylabel('Total Revenue ($K)')
axes[1].set_title('Revenue by Region', fontsize=12, fontweight='bold')

# Revenue per agent
axes[2].bar(regional_analysis['regional_office'], regional_analysis['revenue_per_agent'] / 1000, color=colors)
axes[2].set_ylabel('Revenue per Agent ($K)')
axes[2].set_title('Revenue per Agent by Region', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../visuals/05_regional_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Sales Cycle Analysis

In [ ]:
# Sales cycle duration analysis
cycle_df = closed_deals[closed_deals['days_to_close'].notna()].copy()

cycle_stats = cycle_df.groupby('deal_stage')['days_to_close'].agg(['count', 'mean', 'median', 'std', 'min', 'max'])
cycle_stats = cycle_stats.round(1)

print("=== SALES CYCLE DURATION (days from Engaging to Close) ===")
print(cycle_stats)

In [ ]:
# Visualize sales cycle
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution of days to close
won_cycle = cycle_df[cycle_df['deal_stage'] == 'Won']['days_to_close']
lost_cycle = cycle_df[cycle_df['deal_stage'] == 'Lost']['days_to_close']

axes[0].hist(won_cycle, bins=30, alpha=0.7, label='Won', color='#27ae60')
axes[0].hist(lost_cycle, bins=30, alpha=0.7, label='Lost', color='#e74c3c')
axes[0].set_xlabel('Days to Close')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Sales Cycle Duration', fontsize=14, fontweight='bold')
axes[0].legend()

# Box plot comparison
data_to_plot = [won_cycle, lost_cycle]
bp = axes[1].boxplot(data_to_plot, labels=['Won', 'Lost'], patch_artist=True)
bp['boxes'][0].set_facecolor('#27ae60')
bp['boxes'][1].set_facecolor('#e74c3c')
axes[1].set_ylabel('Days to Close')
axes[1].set_title('Sales Cycle by Outcome', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../visuals/06_sales_cycle_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Key Findings Summary

In [ ]:
print("="*60)
print("KEY FINDINGS SUMMARY")
print("="*60)

print(f"\n1. OVERALL PERFORMANCE")
print(f"   - Overall Win Rate: {overall_win_rate:.1f}%")
print(f"   - Total Revenue: ${won_deals['close_value'].sum():,.0f}")
print(f"   - Average Deal Value: ${won_deals['close_value'].mean():,.0f}")

print(f"\n2. TOP PERFORMING SECTORS")
top_sectors = sector_analysis.nlargest(3, 'win_rate')[['sector', 'win_rate']]
for _, row in top_sectors.iterrows():
    print(f"   - {row['sector']}: {row['win_rate']}% win rate")

print(f"\n3. BEST COMPANY SIZE SEGMENTS")
best_size = size_analysis.nlargest(2, 'win_rate')[['company_size', 'win_rate']]
for _, row in best_size.iterrows():
    print(f"   - {row['company_size']}: {row['win_rate']}% win rate")

print(f"\n4. TOP PRODUCTS BY REVENUE")
top_products = product_analysis.nlargest(3, 'total_revenue')[['product', 'total_revenue', 'win_rate']]
for _, row in top_products.iterrows():
    print(f"   - {row['product']}: ${row['total_revenue']:,.0f} ({row['win_rate']}% win rate)")

print(f"\n5. REGIONAL INSIGHTS")
best_region = regional_analysis.nlargest(1, 'win_rate').iloc[0]
print(f"   - Best performing region: {best_region['regional_office']} ({best_region['win_rate']}% win rate)")

print(f"\n6. SALES CYCLE")
print(f"   - Average days to win: {won_cycle.mean():.0f} days")
print(f"   - Average days to lose: {lost_cycle.mean():.0f} days")

print("\n" + "="*60)